<h1>Scraping Fidelity.com</h1>
In this assignment, you will scrape data from fidelity.com. The goal of the exercise is to get the latest sector performance data from the US markets, and to get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by decreasing order of market weight. I.e., the sector with the highest weight should be in the first tuple, etc.

<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [1]:
def get_us_sector_performance():
    import requests
    from bs4 import BeautifulSoup
    
    # Create an empty list first
    output_list = list()
    
    # request url and apply BeautifulSoup to the content
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"
    response = requests.get(url)
    page_data_soup = BeautifulSoup(response.content,'lxml')
    
    us_sectors = page_data_soup.find_all('a',class_='heading1')
    
    # Loop through each sector
    for sector in us_sectors:
        sector_name = sector.get_text()
        sector_href = sector.get('href')
        sector_link = 'https://eresearch.fidelity.com/' + sector_href
        
        change_cap_weight = get_sector_change_and_market_cap(sector_link)
        sector_change = change_cap_weight[0]
        sector_market_cap = change_cap_weight[1]
        sector_weight = change_cap_weight[2]
        
        output_list.append((sector_name, sector_change, sector_market_cap, sector_weight, sector_link))
    
    # Sort the list in an decreasing order of market weight
    output_list.sort(reverse=True, key=lambda z:z[3])
    return output_list

In [1]:
def get_sector_change_and_market_cap(sector_page_link):
    import requests
    from bs4 import BeautifulSoup
    response = requests.get(sector_page_link)
    page_data_soup = BeautifulSoup(response.content,'lxml')
    header = page_data_soup.find_all('table', class_="snapshot-data-tbl")
    for tags in header:
        # Save the %change as a float.
        try:
            change = tags.find('span',class_='negative').get_text()[:-1] # Slicing to remove %
            sector_change = float(change)
        # Include the exception when the change is positive, because right now all sectors
        # have a negative change and all classes under span shows 'negative'.
        except:
            change = tags.find('span',class_='positive').get_text()[:-1]
            sector_change = float(change)
        
        # Save the market cap as a string.
        sector_market_cap = tags.find_all('td')[1].find('span').get_text()
        
        # Save the market weight as a float.
        weight = tags.find('td',class_='left-spcng').find('span').get_text()[:-1] # Slicing to remove %
        try:
            sector_market_weight = float(weight)
        # Include the exceptions when market weight data is not available or empty
        except ValueError or TypeError:
            sector_market_weight = 0.0

    return sector_change,sector_market_cap,sector_market_weight

In [3]:
#Test get_sector_change_and_market_cap()
link = "https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25"
get_sector_change_and_market_cap(link)
#Should return (-0.40, '$3.58T', 6.80)
#Note that neither the -0.40, nor the 6.80, end with a %sign

(-0.77, '$5.35T', 9.86)

In [4]:
#Test get_us_sector_performance()
get_us_sector_performance()
#Should return (example: obviously the results will vary over time!)

# """
# [('Telecommunication Services',
#   0.21,
#   '$1.74T',
#   2.0,
#   'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
#  ('Materials',
#   0.22,
#   '$1.95T',
#   2.49,
#   'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),
#  ('Real Estate',
#   -0.45,
#   '$1.22T',
#   2.7,
#   'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),
#  ('Utilities',
#   -0.33,
#   '$1.25T',
#   2.86,
#   'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
#  ('Energy',
#   0.76,
#   '$3.90T',
#   5.83,
#   'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),
#  ('Consumer Staples',
#   -0.32,
#   '$3.58T',
#   6.8,
#   'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'),
#  ('Industrials',
#   0.72,
#   '$4.31T',
#   9.83,
#   'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
#  ('Consumer Discretionary',
#   1.03,
#   '$5.76T',
#   12.9,
#   'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
#  ('Financials',
#   0.39,
#   '$7.45T',
#   13.71,
#   'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
#  ('Health Care',
#   0.76,
#   '$5.70T',
#   14.71,
#   'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
#  ('Information Technology',
#   0.81,
#   '$9.84T',
#   26.17,
#   'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45')]
  
# """

[('Information Technology',
  0.79,
  '$8.61T',
  24.32,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),
 ('Health Care',
  -1.41,
  '$5.70T',
  13.89,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Financials',
  -2.57,
  '$6.71T',
  12.49,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Communication Services',
  0.22,
  '$4.57T',
  10.47,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Consumer Discretionary',
  -0.77,
  '$5.35T',
  9.86,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Industrials',
  -1.18,
  '$4.03T',
  8.97,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=

<h3>Sorting</h3>

In [1]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort() #Sorts by the first element of the tuple
x

[('a', 23.2, 'b'), ('c', 17.4, 'f'), ('d', 29.2, 'z'), ('e', 1.74, 'bb')]

In [2]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort(key=lambda k: k[1]) #Sorts by the element at position 1
x

[('e', 1.74, 'bb'), ('c', 17.4, 'f'), ('a', 23.2, 'b'), ('d', 29.2, 'z')]